In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from banditpy.models import BanditTrainer2Arm


bt = BanditTrainer2Arm()
df_train = bt.train(n_sessions=30000, mode="Struc", return_df=True)
bt.save_model()

Starting training for 30000 Structured sessions...


  2%|▏         | 558/30000 [00:40<33:13, 14.77it/s]

In [11]:
from banditpy.core import Bandit2Arm
import matplotlib.pyplot as plt
from neuropy import plotting

df_test = bt.evaluate(mode=[0.7, 0.3], n_sessions=200)

fig = plotting.Fig(6, 5)
ax = fig.subplot(fig.gs[0])
ax.plot(bt.training_loss_history)

for i, df in enumerate([df_train, df_test]):
    ax = fig.subplot(fig.gs[i + 1])
    task = Bandit2Arm(
        probs=df.loc[:, ["arm1_reward_prob", "arm2_reward_prob"]].to_numpy(),
        choices=df["chosen_action"].to_numpy(),
        rewards=df["reward"].to_numpy(),
        session_ids=df["session_id"].to_numpy(),
    )

    ax.plot(task.get_performance())
    ax.set_xlabel("Trials")
    ax.set_ylim(0.4, 1)
    ax.set_ylabel("Pr (High)")

Starting evaluation with fixed weights...
Model and training history loaded from two_arm_task_model.pt
Loaded 2000 training loss values


100%|██████████| 200/200 [00:09<00:00, 20.69it/s]


Evaluation complete.


In [ ]:
isinstance([0.2, 0.2], list)

In [ ]:
type([0, 1]) == list

True

In [ ]:
wt = bt.analyze_weights()
wt

In [ ]:
bt.plot_weight_analysis()

In [ ]:
15991 / 40000

In [ ]:
df_test.shape

In [12]:
df_test

,session_id,chosen_action,reward,arm1_reward_prob,arm2_reward_prob
0,1,2,0.0,0.7,0.3
1,1,2,0.0,0.7,0.3
2,1,2,0.0,0.7,0.3
3,1,1,1.0,0.7,0.3
4,1,1,1.0,0.7,0.3
...,...,...,...,...,...
39995,200,2,0.0,0.7,0.3
39996,200,2,0.0,0.7,0.3
39997,200,2,0.0,0.7,0.3
39998,200,2,0.0,0.7,0.3


In [ ]:
bt._get_reward_probs(True)

In [ ]:
16161 / 200000

In [ ]:
df_test["chosen_action"].unique()

In [ ]:
# ...existing code...

G = self._discounted_return(rewards_received)
x_seq_tensor = torch.stack(input_tensors_for_update).unsqueeze(0)

policy_logits_seq, value_estimates_seq, _ = self.model(x_seq_tensor)
policy_logits_seq = policy_logits_seq.squeeze(0)
value_estimates_seq = value_estimates_seq.squeeze(0)

actions_tensor = torch.tensor(model_actions_taken, dtype=torch.long, device=self.device)

# Calculate log probabilities π(a_t|s_t)
log_probs = torch.distributions.Categorical(logits=policy_logits_seq).log_prob(
    actions_tensor
)

# Calculate advantage A_t = G_t - V(s_t)
advantage = G - value_estimates_seq

# Policy loss: L_policy = -∑ log π(a_t|s_t) * A_t
# Note: We use negative because we want to maximize, but optimizers minimize
policy_loss = -(log_probs * advantage.detach()).mean()

# Value loss: L_value = ∑ (G_t - V(s_t))^2
# This is the squared error between returns and value estimates
value_loss = advantage.pow(2).mean()

# Entropy bonus: H(π) = -∑ π(a|s) * log π(a|s)
# We subtract entropy to encourage exploration (entropy regularization)
policy_dist = torch.distributions.Categorical(logits=policy_logits_seq)
entropy = policy_dist.entropy().mean()

# Total loss: L = L_policy + β_value * L_value - β_entropy * H(π)
# The entropy term is subtracted to encourage exploration
loss = policy_loss + self.beta_value * value_loss - self.beta_entropy * entropy

self.training_loss_history.append(loss.item())
self.optimizer.zero_grad()
loss.backward()
torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
self.optimizer.step()

# ...existing code...